In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy

In [6]:
st = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl') #importing complete stats DB
transStat = copy.deepcopy(st)
transStat.index=list(st.Id)
transStat=transStat.T # transposed df makes it easier to access players by id
#print(transStat.index)
transStat

Index(['Id', 'R', 'Nome', 'Squadra', 'Pg', 'Mv', 'Mf', 'Gf', 'Gs', 'Rp', 'Rc',
       'R+', 'R-', 'Ass', 'Asf', 'Amm', 'Esp', 'Au', 'Qt. A', 'Qt. I', 'Diff.',
       'Classe', 'Eta'', 'Nome Completo', 'Nazionalit'a', 'Nome Squadra',
       'Allenatore'],
      dtype='object')


,2610,785,2531,608,2530,2819,441,26,2764,647,...,4359,4862,4923,2773,5297,5303,2013,4387,4335,4938
Id,2610,785,2531,608,2530,2819,441,26,2764,647,...,4359,4862,4923,2773,5297,5303,2013,4387,4335,4938
R,A,A,A,A,A,A,A,C,A,A,...,A,A,A,A,A,A,A,A,A,A
Nome,RONALDO,IMMOBILE,LUKAKU,ZAPATA D.,IBRAHIMOVIC,CAPUTO,BELOTTI,GOMEZ,MARTINEZ L.,DZEKO,...,PICCOLI,DIALLO,COLOMBO,SPROCATI,LUVUMBO,MALES,RICCI F.,ADORANTE,SCHIAPPACASSE,MORO
Squadra,Juventus,Lazio,Inter,Atalanta,Milan,Sassuolo,Torino,Atalanta,Inter,Roma,...,Spezia,Atalanta,Milan,Parma,Cagliari,Genoa,Sassuolo,Parma,Sassuolo,Lazio
Pg,3,5,5,6,4,5,5,6,6,5,...,3,0,2,1,0,0,1,1,0,0
Mv,7.17,6.1,6.7,6.25,7.38,7,7.2,6.92,6.33,6.3,...,5,0,5.75,5.5,0,0,0,0,0,0
Mf,12.17,7.6,9.6,7.58,11.88,10.4,11,9.25,7.75,8,...,5,0,5.75,5.5,0,0,0,0,0,0
Gf,3,2,5,1,5,4,4,4,3,3,...,0,0,0,0,0,0,0,0,0,0
Gs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Rp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# DB - DICTIONARY SKELETON # 

# Dictionary with specific loan info
LoanDict = {
    'start_date': '2020/08/01',
    'expire_date': '2020/07/31',
    'cost': 0,
    'redemption_right': False, #True or False
    'redemption_cost': 0, #>0 if redemption_right=True
}

# Dictionary with player current team and loan info
CurrentTeamDict = {
    'owner': '', #luca, pietro etc
    'team': '', #team_name 
    'squad': '', # 'Main' or 'Primavera'
    'start_date': '2020/08/01',
    'origin': '', #last team (e.g. team_name + ' - Primavera')
    'quotation_initial': 0,
    'on_loan': False, #True or False
    'loan_info': LoanDict
}

# Dictionary with player ownership info
ContractDict = {
    'owner': '', #this seems redundant
    'start_date': '2020/08/01',
    'cost': 0,
    'acquisition_mode': '', #asta_svincolati, draft, acquisto
    'origin': '', #team_name or svincolati
    'quotation_initial': 0
}

# Dictionary with player personal info
PersonalInfoDict = {
    'full_name': '',
    'birthdate': '1970/01/01',
    'nation': '',
    'team_real': '',
    'FC_role': ''
}

# Nested dictionary for a single player info
PlayerDict = {
    'id': 0,
    'name': '',
    'personal_info': PersonalInfoDict,
    'contract': ContractDict,
    'current_team': CurrentTeamDict,
    #'transfers': TransfersDict,
    #'stats': StatsDict,
    #'seasons': SeasonStatsDict
}

# Main nested dictionary with ALL FPF players info
PlayersDB = {}

In [17]:
# IMPORT CURRENT DATA INTO DB DICTIONARY #
for Id, Name in zip(st['Id'], st['Nome']):
    PlayersDB[Id] = copy.deepcopy(PlayerDict)
    PlayersDB[Id]['id'] = Id
    PlayersDB[Id]['name'] = Name

#print(PlayersDB)

#with open('FPF_Players_Database.json', 'w') as fp:
#    json.dump(PlayersDB, fp)

# Updating entire database with stats and info

Do not run this section unless something is changed in the stats database

In [107]:
def check_prim(text):
    if ' - PRIMAVERA' in text:
        out = text[:-12]
        squad = 'primavera'
    else:
        out = text
        squad = 'main'
    return out, squad

In [108]:
for Id, dic in PlayersDB.items():
    dic['personal_info']['FC_role'] = transStat[Id]['R']
    dic['personal_info']['team_real'] = transStat[Id]['Squadra']
    dic['personal_info']['full_name'] = transStat[Id]['Nome Completo']
    dic['personal_info']['nation'] = transStat[Id]['Nazionalit\'a']
    dic['personal_info']['birthdate'] = transStat[Id]['Classe']
    dic['personal_info']['age'] = transStat[Id]['Eta\'']
    dic['contract']['quotation_initial'] = transStat[Id]['Qt. I']
    dic['contract']['owner'] = transStat[Id]['Allenatore']
    dic['current_team']['owner'] = transStat[Id]['Allenatore']
    dic['current_team']['team'], dic['current_team']['squad']  = check_prim(transStat[Id]['Nome Squadra'])
    dic['current_team']['quotation_initial'] = transStat[Id]['Qt. I']
    dic['stats'] = dict(transStat[Id][4:18])

# COMMENTED TO PREVENT OVERWRITING    
#with open('FPF_Players_Database.json', 'w') as fp:
#    json.dump(PlayersDB, fp)

In [109]:
# WRITE PLAYER IDS TO TXT FILE #
f = open("PlayersIds.txt", "w")
f.write('Id Name\n')
for Id,Name in zip(st['Id'], st['Nome']):
    f.write(str(Id)+' '+Name+'\n')
f.close()

# Import created database and upload it to mongoDB

## Note: I need to fix the credentials. For now I have to add the ip address of the machine connecting to the db, so it is not ideal because I'm not using static ip 

In [8]:
from pymongo import MongoClient
from pprint import pprint
import pymongo

with open('FPF_Players_Database.json','r') as f:
    PlayersDB_in = json.load(f)


with open('credential.json','r') as f:
    cred = json.load(f)

In [9]:
#Creating a list of dictionaries, which is the preferred structure for MongoDB
posts = []
for idx, dic in PlayersDB_in.items():
    #Note: the change to label '_id' is to respect MongoDB standards
    temp = {"_id": dic['id'], "name": dic['name'],"owner": dic['current_team']['owner'],"team": dic['current_team']['team'],"squad": dic['current_team']['squad'],"real_team": dic['personal_info']['team_real'],"role": dic['personal_info']['FC_role'],"info": dic}
                                         
    posts.append(temp)

In [10]:
#Inserting database for the first time
# specifying cluster with credential
with open('credential.json','r') as f:
    PlayersDB_in = json.load(f)

    
cluster = MongoClient(cred['cred'])
# choosing database
db = cluster["Game"]
# choosing collection
collection = db["Players"]

""" this is to fill the collection with the entire database, 
    but it can't be used to overwrite, so I used it only the first time
    and then commented """

#collection.insert_many(posts)

" this is to fill the collection with the entire database, \n    but it can't be used to overwrite, so I used it only the first time\n    and then commented "

### Every entry in the database can be queried by external keys

Note: mongoDB allows you to query (either to find, delete or replace) by the keys in the outmost dictionary, not the nested dictionary, so we need to decide what we want to have as queriable keys and flatten those.

We can query by:

- id (key '_id') this follows mongoDB standards and it MUST BE unique
- name
- owner
- team
- squad
- real_team
- role

We can think whether to add more queriable keys, for instance:

- birthdate? (maybe just year)

In MongoDB you can easily cross these queries and, for instance, find all the players that match multiple criteria (examples below)

In [11]:
#Example of a find query by name: 
#find the element that matches the criteria
#Note: it makes sense if you search unique elements, 
#if you search by team it gives only first result, in that case see below
result = collection.find_one({"name":'MUSSO'})

pprint(result) #better way to print dictionaries

{'_id': 2792,
 'info': {'contract': {'acquisition_mode': '',
                       'cost': 0,
                       'origin': '',
                       'owner': 'pietro',
                       'quotation_initial': 11,
                       'start_date': '2020/08/01'},
          'current_team': {'loan_info': {'cost': 0,
                                         'expire_date': '2020/07/31',
                                         'redemption_cost': 0,
                                         'redemption_right': False,
                                         'start_date': '2020/08/01'},
                           'on_loan': False,
                           'origin': '',
                           'owner': 'pietro',
                           'quotation_initial': 11,
                           'squad': 'main',
                           'start_date': '2020/08/01',
                           'team': 'PDG 1908'},
          'id': 2792,
          'name': 'MUSSO',
          'personal_inf

In [12]:
#Example, single criteria with multiple matches
#Find all Atalanta players

results = collection.find({'real_team': 'Atalanta'})
for result in results:
    print(result['name']) #printing just names for convenience

ZAPATA D.
GOMEZ
MURIEL
GOSENS
ILICIC
MALINOVSKYI
PASALIC
MIRANCHUK
HATEBOER
LAMMERS
GOLLINI
TOLOI
FREULER
DE ROON
PALOMINO
DJIMSITI
ROMERO
PICCINI
PESSINA
CALDARA
DEPAOLI
MOJICA
SPORTIELLO
SUTALO
CARNESECCHI
ROSSI F.
RADUNOVIC
DIALLO


In [11]:
#Example of multiple criteria: 
#Find all Lazio players in pietro's team

results = collection.find({'owner': 'pietro', 'real_team': 'Lazio'}) #query order is irrelevant

#it returns a cursor, so you need to loop over it to see the content
for result in results:
    print(result['name']) #printing just names for convenience

LUIS ALBERTO
LAZZARI
MARUSIC


In [5]:
#Example of multiple criteria: 
#Find all main squad players in pietro's team

results = collection.find({'owner': 'pietro', 'squad': 'main'})
for result in results:
    print(result['name']) #printing just names for convenience

ZAPATA D.
MURIEL
LUIS ALBERTO
CALHANOGLU
CHIESA
HERNANDEZ T.
DE PAUL
RAFAEL LEAO
POLITANO
LOZANO
GABBIADINI
MANOLAS
BONAZZOLI
LAZZARI
SPINAZZOLA
SENSI
DRAGOWSKI
MUSSO
D'AMBROSIO
CANDREVA
GLIK
SKRINIAR
TOLJAN
MONTIPO'
PELLEGRINI LU.


In [15]:
#Example of multiple criteria: 
#Find all primavera squad players in luca's team who play for Verona

results = collection.find({'owner': 'luca', 'squad': 'primavera', 'real_team': 'Verona'})
for result in results:
    print(result['name']) #printing just names for convenience

FAVILLI
SALCEDO E.


### Deleting elements from the database

I left every cell commented to avoid erasing elements by mistake

In [65]:
#deleting a single element that matches the criteria: (DON'T RUN)

#THIS deletes the entire entry that matches the chosen criteria (name)
# results = collection.delete_one({"name": "MUSSO"})

In [ ]:
# THIS deletes every entry that matches this criteria (we don't have duplicates, so it is always one)
#
# results = collection.delete_many({"name": "MUSSO"})

In [118]:
# USE THIS ONLY IF YOU WANT TO DELETE ALL:

#results = collection.delete_many({})

### Update elements in the database

The general command is the following 

```collection.update_many(<criteria>, {$set:{<key>: <new value>}})
```
    
Since we have a nested dictionary and we need to modify multiple entries at the same time (e.g. owner and team ) the best logic to follow is: 
- to download the entire entry using find and assign it to a dictionary
- modify the entire dictionary
- update the entire element

There are several ways of doing it but we definitely need 2 minimal components:
- One function that given the dictionary "info" returns an updated element in the form of the content of the array 'posts' defined above
- one function that downloads the info database (using whatever query you want), calls the previous function to create the new element

In any case, we need to make the functions as modular as possible, so that we need do define only a few

### Access to nested keys via dot notation 
We can update any element of the nested disctionaries

In [24]:
# Example, Updating a nested key 
collection.update_one({'name': 'MUSSO'}, {'$set':{'info.contract.acquisition_mode': 'Scambio'}})
result = collection.find_one({"name":'MUSSO'})
pprint(result) #better way to print dictionaries

{'_id': 2792,
 'info': {'contract': {'acquisition_mode': 'Scambio',
                       'cost': 0,
                       'origin': '',
                       'owner': 'pietro',
                       'quotation_initial': 11,
                       'start_date': '2020/08/01'},
          'current_team': {'loan_info': {'cost': 0,
                                         'expire_date': '2020/07/31',
                                         'redemption_cost': 0,
                                         'redemption_right': False,
                                         'start_date': '2020/08/01'},
                           'on_loan': False,
                           'origin': '',
                           'owner': 'pietro',
                           'quotation_initial': 11,
                           'squad': 'main',
                           'start_date': '2020/08/01',
                           'team': 'PDG 1908'},
          'id': 2792,
          'name': 'MUSSO',
          'perso

In [25]:
#Example, find elements by nested key query

results = collection.find({'info.current_team.quotation_initial': 11})
for result in results:
    print(result['name']) #printing just names for convenience

LOZANO
CAPRARI
SILVESTRI
HATEBOER
LAZZARI
JANKTO
DEFREL
LAMMERS
CHIELLINI
FREULER
BARAK
PEREYRA
CRAGNO
MUSSO
N'KOULOU
SMALLING
ZAJC
PINAMONTI
SIRIGU
DI CARMINE


# In the following section we define the functions that we need to manipulate the database